In [1]:
import pandas as pd
import re
import unicodedata
import numpy as np
import datetime

# Importante: para todos os notebooks não estou usando o arquivo inicial que elas mandaram
# já substituí a aba "Comercial" pelos dados corretos (que vieram com o nome "Volume com regionais")

# Ajustar o caminho conforme necessário
FILE_PATH = "data/dados.xlsx"
SHEET_NAME = "Matéria-Prima"

# ===================================================================
# 1. Carregar usando header=2 (a linha 'jan/25'...)
# ===================================================================
try:
    df_mp_raw = pd.read_excel(
        FILE_PATH, 
        sheet_name=SHEET_NAME, 
        header=2 # Usar a 3ª linha (índice 2) como cabeçalho
    )
    print("1. Planilha 'Matéria-Prima' carregada (header=2)")
    
except Exception as e:
    print(f"ERRO ao carregar a planilha: {e}")
    raise e

# ===================================================================
# 2. Renomear Colunas de ID
# ===================================================================
df_mp = df_mp_raw.copy()

# Nomes de ID fixos
id_cols_clean = ['id_planta', 'item_materia_prima', 'unidade_medida']

# Renomear as 3 primeiras colunas pela posição
try:
    df_mp = df_mp.rename(columns={
        df_mp.columns[0]: id_cols_clean[0],
        df_mp.columns[1]: id_cols_clean[1],
        df_mp.columns[2]: id_cols_clean[2] 
    })
    print("\n2. Colunas de ID renomeadas")
    
except Exception as e:
    print(f"ERRO ao renomear colunas: {e}")
    raise

# ===================================================================
# 3. Remover Linha de Lixo
# ===================================================================
df_mp_data = df_mp.iloc[1:].copy() # Remover a linha 0 ('Jundiaí...')
df_mp_data['id_planta'] = df_mp_data['id_planta'].str.strip()
df_mp_data['item_materia_prima'] = df_mp_data['item_materia_prima'].str.strip()

print("\n3. Linha de lixo removida")

# ===================================================================
# 4. Fazer o "Unpivot" (Melt)
# ===================================================================
value_vars_dates = [
    col for col in df_mp_data.columns 
    if isinstance(col, (datetime.datetime, pd.Timestamp))
]

print(f"\n4. {len(value_vars_dates)} Colunas de Data identificadas para o Unpivot")

df_mp_clean = pd.melt(
    df_mp_data,
    id_vars=id_cols_clean,
    value_vars=value_vars_dates,
    var_name='depara_mess', # A coluna é criada como object
    value_name='custo_driver_projetado_str'
)

# ===================================================================
# 5. Limpeza Final (Decimais e Datas)
# ===================================================================

# Forçar a coluna 'depara_mess' (que o melt criou como object) de volta para datetime
df_mp_clean['depara_mess'] = pd.to_datetime(
    df_mp_clean['depara_mess'], 
    errors='coerce'
)

# Trocar vírgula por ponto
df_mp_clean['custo_driver_projetado'] = (
    df_mp_clean['custo_driver_projetado_str']
    .astype(str)
    .str.replace(",", ".") # Troca a vírgula do decimal
)

# Converter para numérico
df_mp_clean['custo_driver_projetado'] = pd.to_numeric(
    df_mp_clean['custo_driver_projetado'], 
    errors='coerce'
)

# Limpar a coluna 'item' (remover prefixo 'JD' e espaços)
df_mp_clean['item_materia_prima'] = (
    df_mp_clean['item_materia_prima']
    .str.replace(r"^JD", "", regex=True)
    .str.strip()
)

# Filtro final
df_mp_clean = df_mp_clean[
    (df_mp_clean['custo_driver_projetado'].notna()) & 
    (df_mp_clean['custo_driver_projetado'] != 0) &   
    (df_mp_clean['depara_mess'].notna()) # Agora este filtro funciona
].copy()

# Selecionar e reordenar as colunas finais
final_cols_clean = [
    'depara_mess', 
    'id_planta', 
    'item_materia_prima', 
    'unidade_medida', 
    'custo_driver_projetado'
]
df_mp_clean = df_mp_clean[final_cols_clean]

print("\n--- 'Matéria-Prima' Limpa e Despivotada (FINAL) ---")
display(df_mp_clean.head())
print(f"Shape final: {df_mp_clean.shape}")
print(f"Tipo de 'depara_mess': {df_mp_clean['depara_mess'].dtype}") # Deve ser datetime64[ns]

if df_mp_clean.shape[0] > 0:
    print(f"Período: {df_mp_clean['depara_mess'].min().date()} a {df_mp_clean['depara_mess'].max().date()}")
else:
    print("Período: Nenhum dado encontrado após a limpeza.")

1. Planilha 'Matéria-Prima' carregada (header=2)

2. Colunas de ID renomeadas

3. Linha de lixo removida

4. 12 Colunas de Data identificadas para o Unpivot

--- 'Matéria-Prima' Limpa e Despivotada (FINAL) ---


,depara_mess,id_planta,item_materia_prima,unidade_medida,custo_driver_projetado
0,2025-01-01,JD,ITEM,PTAX,5.830100
1,2025-01-01,JDLata 7.5oz (220ml),Lata 7.5oz (220ml),R$/Mh,19.325093
2,2025-01-01,JDLata 10.5oz (310ml),Lata 10.5oz (310ml),R$/Mh,23.073173
3,2025-01-01,JDLata 12oz (350ml),Lata 12oz (350ml),R$/Mh,22.806567
4,2025-01-01,"JDGarrafa OWG 9,8oz (descartável 290ml)","Garrafa OWG 9,8oz (descartável 290ml)",R$/Mh,89.375782


Shape final: (276, 5)
Tipo de 'depara_mess': datetime64[ns]
Período: 2025-01-01 a 2025-12-01


In [2]:
import pandas as pd
import re
import unicodedata
import numpy as np

# 1. Funções de Limpeza (reutilizadas)
def clean_colname(col: str) -> str:
    col = unicodedata.normalize("NFKD", str(col))
    col = "".join(c for c in col if not unicodedata.combining(c))
    col = col.strip().lower()
    col = re.sub(r"\s+", "_", col)          # espaços -> _
    col = re.sub(r"[^\w_]", "", col)        # remove caracteres especiais
    return col

# 2. Carregar dados brutos
FILE_PATH = "data/dados.xlsx" 
SHEET_NAME_FIN = "Financeiro" # Mesmo arquivo, muda a aba

try:
    df_fin_raw = pd.read_excel(FILE_PATH, sheet_name=SHEET_NAME_FIN)
    print(f"1. Aba '{SHEET_NAME_FIN}' Carregada (raw)")
    print(f"Shape original: {df_fin_raw.shape}")
except Exception as e:
    print(f"ERRO ao carregar a aba '{SHEET_NAME_FIN}': {e}")
    raise e

# 3. Limpeza de nomes e textos
df_fin_clean = df_fin_raw.copy()
df_fin_clean.columns = [clean_colname(c) for c in df_fin_clean.columns]

# Colunas de texto que formam nossas chaves
text_key_cols = [
    "diretoria", "marca", "tipo_consumo", 
    "retornabilidade", "estado", "regional", "embalagem"
]

for c in text_key_cols:
    df_fin_clean[c] = (
        df_fin_clean[c]
        .astype(str)
        .str.strip()
        .str.upper()
        .str.replace(r"\s+", " ", regex=True)
    )

# 4. Harmonização de chaves

# 4a. Embalagem (Lata -> Alumínio, etc.)
embalagem_map = {
    "VIDRO": "VIDRO",
    "LATA": "ALUMINIO",
    "BAG IN BOX": "BAG IN BOX",
    "REF PET": "PET",
    "PET": "PET"
}
df_fin_clean["embalagem"] = df_fin_clean["embalagem"].map(embalagem_map)

# 4b. Mapa de Marca 
# Aqui ia entrar o de-para de marca, mas como não consegui fazer bater, achei melhor deixar como está
# Caso queira adicionar um mapeamento, descomente e ajuste o código abaixo:
# marca_map = {
#     "CC": "Coca-Cola",
#     "CC IMPORT": "Coca-Cola Importada",
#     "CC MENOS AÇÚ": "Coca-Cola Menos Açúcar"
# }
# df_fin_clean["marca"] = df_fin_clean["marca"].map(marca_map)

# 5. Criação da Chave-Mestra (SKU)
df_fin_clean["chave_sku"] = (
    df_fin_clean["marca"].astype(str) + "|" + 
    df_fin_clean["tamanho"].astype(float).astype(str) + "|" + 
    df_fin_clean["tipo_consumo"].astype(str) + "|" +
    df_fin_clean["retornabilidade"].astype(str) + "|" + 
    df_fin_clean["embalagem"].astype(str)
)

print("\n2. Chaves Harmonizadas e 'chave_sku' criada")
print(f"SKUs únicos no Financeiro (não agregados): {df_fin_clean['chave_sku'].nunique()}")

# 6. Agregação (para eliminar duplicatas)
CHAVE_FINAL_AGG = ["depara_mess", "regional", "diretoria", "chave_sku"]

metric_cols = [
    'volume', 'gross_revenue', 'taxes', 'discounts', 'encargos', 
    'net_revenue', 'other_revenue', 'total_revenue', 
    'cvv_concentrate', 'cvv_sweetener', 'cvv_pet', 'cvv_can', 'cvv_cap', 
    'cvv_purcharses', 'cvv_otherraw', 'cvv_total', 'gvv_labor', 'gvv_t1', 
    'gvv_t2', 'gvv_ed', 'gvv_othersgvv', 'gvv_total', 'variable_margin', 'dme'
]

print("\n3. Agregando dados para garantir unicidade...")
df_fin_agg = (
    df_fin_clean.groupby(CHAVE_FINAL_AGG)
    .agg({col: 'sum' for col in metric_cols})
    .reset_index()
)

print(f"Shape após agregação: {df_fin_agg.shape}")
print(f"Duplicatas restantes: {df_fin_agg.duplicated(subset=CHAVE_FINAL_AGG).sum()}")

# 7. Cálculo das métricas unitárias (P&L)
df_fin_pnl = df_fin_agg.copy()

# A. Métricas de receita
df_fin_pnl['preco_bruto_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['gross_revenue'] / df_fin_pnl['volume'], 0
)
df_fin_pnl['preco_liquido_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['net_revenue'] / df_fin_pnl['volume'], 0
)

# B. Métricas de custo variável (CVV)
df_fin_pnl['cvv_total_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['cvv_total'] / df_fin_pnl['volume'], 0
)
df_fin_pnl['cvv_pet_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['cvv_pet'] / df_fin_pnl['volume'], 0
)
df_fin_pnl['cvv_can_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['cvv_can'] / df_fin_pnl['volume'], 0
)
df_fin_pnl['cvv_sweetener_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['cvv_sweetener'] / df_fin_pnl['volume'], 0
)

# C. Métricas de custo fixo/variável (GVV)
df_fin_pnl['gvv_total_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['gvv_total'] / df_fin_pnl['volume'], 0
)
df_fin_pnl['gvv_labor_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['gvv_labor'] / df_fin_pnl['volume'], 0
)

# D. Métricas de margem
df_fin_pnl['margem_variavel_unit'] = np.where(
    df_fin_pnl['volume'] > 0, 
    df_fin_pnl['variable_margin'] / df_fin_pnl['volume'], 0
)

print("\n4. Métricas Unitárias Calculadas")

colunas_pnl_final = CHAVE_FINAL_AGG + [
    'volume', 'preco_bruto_unit', 'preco_liquido_unit', 'cvv_total_unit',
    'cvv_pet_unit', 'cvv_can_unit', 'cvv_sweetener_unit', 'gvv_total_unit',
    'gvv_labor_unit', 'margem_variavel_unit'
]

# DATAFRAME DE P&L HISTÓRICO
df_fin_pnl_final = df_fin_pnl[colunas_pnl_final]

display(df_fin_pnl_final.head())
print(f"Shape final do P&L: {df_fin_pnl_final.shape}")

1. Aba 'Financeiro' Carregada (raw)
Shape original: (26743, 34)

2. Chaves Harmonizadas e 'chave_sku' criada
SKUs únicos no Financeiro (não agregados): 101

3. Agregando dados para garantir unicidade...
Shape após agregação: (26743, 28)
Duplicatas restantes: 0

4. Métricas Unitárias Calculadas


,depara_mess,regional,diretoria,chave_sku,volume,preco_bruto_unit,preco_liquido_unit,cvv_total_unit,cvv_pet_unit,cvv_can_unit,cvv_sweetener_unit,gvv_total_unit,gvv_labor_unit,margem_variavel_unit
0,2023-01-01,TERRIT. MG,EVENTUAIS,CC CAFE|0.22|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,45.3600,3.590035,2.557937,2.064683,0.000000,1.168386,0.070326,0.830820,0.453483,-0.131437
1,2023-01-01,TERRIT. MG,EVENTUAIS,CC MENOS AÇÚ|2.0|FUTURO (MS)|DESCARTAVEL|PET,6186.5550,2.013474,1.142688,0.520780,0.127582,0.000000,0.108319,0.210165,0.032823,0.576913
2,2023-01-01,TERRIT. MG,EVENTUAIS,CC MENOS AÇÚ|2.25|FUTURO (MS)|DESCARTAVEL|PET,25.8750,2.019324,1.439459,0.584116,0.142841,0.000000,0.108831,0.077372,0.005101,0.983729
3,2023-01-01,TERRIT. MG,EVENTUAIS,CC MENOS AÇÚ|2.5|FUTURO (MS)|DESCARTAVEL|PET,476.5725,1.735220,1.244092,0.531520,0.123394,0.000000,0.108298,0.149891,0.022481,0.768937
4,2023-01-01,TERRIT. MG,EVENTUAIS,CC MENOS AÇÚ|3.0|FUTURO (MS)|DESCARTAVEL|PET,173.5875,1.559375,1.117949,0.497444,0.115077,0.000000,0.108234,0.129906,0.020151,0.696801


Shape final do P&L: (26743, 14)


In [3]:
import pandas as pd
import re
import numpy as np

# 1. Definir Funções Utilitárias

# Função para extrair Litros (reutilizada)
OZ_TO_ML = 29.5735
def parse_liters(s: str) -> float:
    if not isinstance(s, str):
        s = str(s)
    txt = s.upper()

    m_l = re.search(r"(\d+(?:[\.,]\d+)?)\s*LITRO", txt)
    if m_l:
        val = float(m_l.group(1).replace(",", "."))
        return val

    m_ml = re.search(r"(\d+(?:[\.,]\d+)?)\s*ML", txt)
    if m_ml:
        ml = float(m_ml.group(1).replace(",", "."))
        return ml / 1000.0

    m_oz = re.search(r"(\d+(?:[\.,]\d+)?)\s*(ONZ|OZ|ONCA|ONCAS)", txt)
    if m_oz:
        oz = float(m_oz.group(1).replace(",", "."))
        return (oz * OZ_TO_ML) / 1000.0

    return np.nan

# 2. Carregar Planilha de Elasticidade
FILE_PATH_ELAST = "data/Elasticidade_Envio.xlsx"

try:
    df_elast_raw = pd.read_excel(FILE_PATH_ELAST)
    print(f"1. Arquivo '{FILE_PATH_ELAST}' Carregado")
    display(df_elast_raw.head(3))
except FileNotFoundError:
    print(f"ERRO: Arquivo '{FILE_PATH_ELAST}' não encontrado.")
    raise
except Exception as e:
    print(f"ERRO ao carregar a planilha de elasticidade: {e}")
    raise e

# 3. Limpeza e Harmonização
df_elast_clean = df_elast_raw.copy()
df_elast_clean.columns = [clean_colname(c) for c in df_elast_clean.columns]

text_cols = ['marca', 'embalagem', 'retornabilidade', 'tipo_consumo', 'tamanho']
for c in text_cols:
    df_elast_clean[c] = (
        df_elast_clean[c]
        .astype(str)
        .str.strip()
        .str.upper()
        .str.replace(r"\s+", " ", regex=True)
    )

# 4. Aplicar Mapas (SEM Agregação de Marca)

# 4a. Mapa de Embalagem (Lata -> Alumínio, etc.)
embalagem_map = {
    "VIDRO": "VIDRO",
    "LATA": "ALUMINIO",
    "BAG IN BOX": "BAG IN BOX",
    "REF PET": "PET",
    "PET": "PET",
    "ALUMINIO": "ALUMINIO"
}
df_elast_clean["embalagem"] = df_elast_clean["embalagem"].map(embalagem_map)

# 4b. Mapa de Marca
# Aqui ia entrar o de-para de marca, mas como não consegui fazer bater, achei melhor deixar como está
# Caso queira adicionar um mapeamento, descomente e ajuste o código abaixo:
# marca_map = {         
#     "CC": "Coca-Cola",
#     "CC IMPORT": "Coca-Cola Importada",
#     "CC MENOS AÇÚ": "Coca-Cola Menos Açúcar"
# }
# df_elast_clean["marca"] = df_elast_clean["marca"].map(marca_map)

# 4c. Normalizar 'tipo_consumo'
df_elast_clean["tipo_consumo"] = (
    df_elast_clean["tipo_consumo"]
    .str.replace(r"^\s*CONSUMO\s+", "", regex=True)
    .str.replace(r"IMEDIATO\s*\(?\s*SS\)?", "IMEDIATO (SS)", regex=True)
    .str.replace(r"FUTURO\s*\(?\s*MS\)?", "FUTURO (MS)", regex=True)
)

# 4d. Extrair Tamanho (L)
df_elast_clean['tamanho_l'] = df_elast_clean['tamanho'].apply(parse_liters)

# 5. Criar Chave-Mestra (SKU)
df_elast_clean["chave_sku"] = (
    df_elast_clean["marca"].astype(str) + "|" + 
    df_elast_clean["tamanho_l"].astype(float).astype(str) + "|" + 
    df_elast_clean["tipo_consumo"].astype(str) + "|" +
    df_elast_clean["retornabilidade"].astype(str) + "|" + 
    df_elast_clean["embalagem"].astype(str)
)

# 6. Limpeza e Validação Final
df_elast_clean['elasticidade'] = (
    df_elast_clean['elasticidade']
    .astype(str)
    .str.replace(",", ".")
)
df_elast_clean['elasticidade'] = pd.to_numeric(
    df_elast_clean['elasticidade'],
    errors='coerce'
)

df_elasticidade_final = df_elast_clean[['chave_sku', 'elasticidade']].copy()
df_elasticidade_final = df_elasticidade_final.dropna()

dups_elast = df_elasticidade_final.duplicated(subset=['chave_sku']).sum()
if dups_elast > 0:
    print(f"ATENÇÃO: Encontradas {dups_elast} duplicatas de SKU na Elasticidade. Usando a média.")
    df_elasticidade_final = (
        df_elasticidade_final.groupby('chave_sku')
        .agg(elasticidade=('elasticidade', 'mean'))
        .reset_index()
    )

print("\n3. Planilha de Elasticidade Harmonizada")
display(df_elasticidade_final.head())
print(f"Shape final: {df_elasticidade_final.shape}")

# 7. Validar Interseção com P&L
skus_pnl = set(df_fin_pnl_final['chave_sku'].unique())
skus_elast = set(df_elasticidade_final['chave_sku'].unique())

intersecao = skus_pnl.intersection(skus_elast)
skus_pnl_sem_elasticidade = skus_pnl - skus_elast

print("\n--- 4. Validação de Chaves (Elasticidade vs P&L) ---")
print(f"SKUs únicos no P&L Histórico: {len(skus_pnl)}")
print(f"SKUs únicos na Elasticidade: {len(skus_elast)}")
print(f"SKUs em COMUM (interseção): {len(intersecao)}")
print(f"SKUs do P&L que NÃO têm elasticidade: {len(skus_pnl_sem_elasticidade)}")

if len(skus_pnl_sem_elasticidade) > 0:
    print("\nAmostra de SKUs do P&L sem elasticidade:")
    print(list(skus_pnl_sem_elasticidade)[:5])

1. Arquivo 'data/Elasticidade_Envio.xlsx' Carregado


,Categoria BR,Sub Categoria,Marca,Embalagem,Tamanho,Retornabilidade,Tipo Consumo,Elasticidade
0,REFRIG,COLAS,CC,VIDRO,20.3 Onz / 600 ML=CC,RETORNAVEL,CONSUMO IMEDIATO (SS,-2.185018
1,REFRIG,COLAS,CC,VIDRO,33.8 Onz / 1 LITRO,RETORNAVEL,CONSUMO FUTURO (MS),-1.129191
2,REFRIG,COLAS,CC,VIDRO,"250ml / 8,5oz",DESCARTAVEL,CONSUMO IMEDIATO (SS,-1.300000


ATENÇÃO: Encontradas 88 duplicatas de SKU na Elasticidade. Usando a média.

3. Planilha de Elasticidade Harmonizada


,chave_sku,elasticidade
0,120|0.75|FUTURO (MS)|DESCARTAVEL|VIDRO,-1.487895
1,1906|0.33|IMEDIATO (SS)|DESCARTAVEL|VIDRO,-1.385810
2,1906|0.354|IMEDIATO (SS)|DESCARTAVEL|VIDRO,-1.300000
3,3 MEDALHAS|0.75|FUTURO (MS)|DESCARTAVEL|VIDRO,-1.198112
4,7BELO|nan|NAO BEBIDA|NAO BEBIDA|nan,-1.869376


Shape final: (331, 2)

--- 4. Validação de Chaves (Elasticidade vs P&L) ---
SKUs únicos no P&L Histórico: 101
SKUs únicos na Elasticidade: 331
SKUs em COMUM (interseção): 44
SKUs do P&L que NÃO têm elasticidade: 57

Amostra de SKUs do P&L sem elasticidade:
['FANTAUVA|1.0|FUTURO (MS)|RETORNAVEL|VIDRO', 'FANTALAR|0.35|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO', 'CC MENOS AÇÚ|2.25|FUTURO (MS)|DESCARTAVEL|PET', 'CC MENOS AÇÚ|3.0|FUTURO (MS)|DESCARTAVEL|PET', 'CC SEM AÇÚ|5.0|IMEDIATO (SS)|DESCARTAVEL|BAG IN BOX']


In [4]:
import pandas as pd
import re
import numpy as np
import unicodedata

# 1. Carregar dados da projeção
FILE_PATH = "data/dados.xlsx"
SHEET_NAME_PROJ = "Projeção"

try:
    # header=3 (a 4ª linha) é o cabeçalho correto
    df_proj_raw = pd.read_excel(
        FILE_PATH, 
        sheet_name=SHEET_NAME_PROJ, 
        header=3 
    )
    print(f"1. Aba '{SHEET_NAME_PROJ}' Carregada")
    display(df_proj_raw.head(3))
except Exception as e:
    print(f"ERRO ao carregar a aba '{SHEET_NAME_PROJ}': {e}")
    raise e

# 2. Limpeza e unpivot
df_proj = df_proj_raw.copy()

# Definir as colunas de ID (como elas aparecem no excel)
id_cols_raw = [
    'Marca', 'Tamanho', 'Retornabilidade', 
    'Diretoria Estratégica'
]

# Definir as colunas de data (todas que NÃO são de ID)
date_cols = [col for col in df_proj.columns if col not in id_cols_raw]

# Fazer o "unpivot" primeiro, usando os nomes originais
df_proj_clean = pd.melt(
    df_proj,
    id_vars=id_cols_raw,
    value_vars=date_cols,
    var_name='depara_mess_str',
    value_name='volume_projetado'
)

# Renomear e limpar as colunas de ID
df_proj_clean = df_proj_clean.rename(columns={
    'Marca': 'marca',
    'Tamanho': 'tamanho',
    'Retornabilidade': 'retornabilidade',
    'Diretoria Estratégica': 'diretoria_estrategica'
})

print("\n2. Planilha 'Projeção' Despivotada")

# 3. Harmonização de Chaves

# 3a. Converter Datas
df_proj_clean['depara_mess'] = pd.to_datetime(
    df_proj_clean['depara_mess_str'],
    errors='coerce' # As datas já são objetos datetime, só converter
)

# 3b. Limpar colunas de texto
text_cols = ['marca', 'tamanho', 'retornabilidade', 'diretoria_estrategica']
for c in text_cols:
    df_proj_clean[c] = (
        df_proj_clean[c]
        .astype(str)
        .str.strip()
        .str.upper()
        .str.replace(r"\s+", " ", regex=True)
    )

# 3c. Harmonizar marcas
df_proj_clean['marca'] = df_proj_clean['marca'].replace("COCA-COLA", "CC")

# 3d. Harmonizar Tamanho (ex: '1,5L' -> 1.5)
def parse_tamanho_projecao(s: str) -> float:
    if not isinstance(s, str):
        return np.nan
    
    match = re.search(r"(\d+(?:[\.,]\d+)?)\s*L", s)
    if match:
        val = float(match.group(1).replace(",", "."))
        return val
    
    match_ml = re.search(r"(\d+)\s*ML", s)
    if match_ml:
        ml = float(match_ml.group(1))
        return ml / 1000.0
    
    return np.nan

df_proj_clean['tamanho_l'] = df_proj_clean['tamanho'].apply(parse_tamanho_projecao)

# 3e. Harmonizar diretoria
diretoria_map = {
    'DISTRIBUIDORES': 'DISTRIBUIDORES DE AREA',
    'HOME MARKET': 'HOME MARKET',
    'ON PREMISE': 'ON PREMISE'
}
df_proj_clean['diretoria'] = df_proj_clean['diretoria_estrategica'].map(diretoria_map)

# 4. Criar Chave-Mestra Parcial
df_proj_clean['chave_sku_parcial'] = (
    df_proj_clean["marca"].astype(str) + "|" + 
    df_proj_clean["tamanho_l"].astype(float).astype(str) + "|" +
    df_proj_clean["retornabilidade"].astype(str)
)

# 5. Limpeza Final (CORRIGIDA)
df_proj_clean['volume_projetado'] = pd.to_numeric(
    df_proj_clean['volume_projetado'], 
    errors='coerce'
)

# Aplicar filtros
df_proj_clean = df_proj_clean[
    (df_proj_clean['depara_mess'].notna()) &
    (df_proj_clean['tamanho_l'].notna()) &
    (df_proj_clean['diretoria'].notna()) &
    (df_proj_clean['volume_projetado'].notna()) &
    (df_proj_clean['volume_projetado'] != 0)
].copy()

# Selecionar colunas finais
colunas_proj_final = [
    'depara_mess',
    'diretoria',
    'chave_sku_parcial',
    'volume_projetado'
]

df_projecao_final = df_proj_clean[colunas_proj_final]

print("\n3. Planilha de Projeção Harmonizada (Corrigida)")
display(df_projecao_final.head())
print(f"Shape final: {df_projecao_final.shape}")

1. Aba 'Projeção' Carregada


,Marca,Tamanho,Retornabilidade,Diretoria Estratégica,2025-06-01 00:00:00,2025-07-01 00:00:00,2025-08-01 00:00:00,2025-09-01 00:00:00,2025-10-01 00:00:00,2025-11-01 00:00:00,2025-12-01 00:00:00
0,COCA-COLA,"1,5L",DESCARTAVEL,Distribuidores,24550.660334,17284.325333,18142.621635,19011.267382,22037.449727,17480.528992,20695.562774
1,COCA-COLA,"1,5L",DESCARTAVEL,Home Market,647103.945245,667138.498602,676759.330712,717081.563768,781575.922764,830372.538418,818841.043617
2,COCA-COLA,"1,5L",DESCARTAVEL,On Premise,50646.771909,63396.131051,67960.714831,68806.343864,73596.458200,65595.240935,66482.761223



2. Planilha 'Projeção' Despivotada

3. Planilha de Projeção Harmonizada (Corrigida)


,depara_mess,diretoria,chave_sku_parcial,volume_projetado
0,2025-06-01,DISTRIBUIDORES DE AREA,CC|1.5|DESCARTAVEL,24550.660334
1,2025-06-01,HOME MARKET,CC|1.5|DESCARTAVEL,647103.945245
2,2025-06-01,ON PREMISE,CC|1.5|DESCARTAVEL,50646.771909
4,2025-06-01,DISTRIBUIDORES DE AREA,CC|10.0|DESCARTAVEL,4793.266378
5,2025-06-01,HOME MARKET,CC|10.0|DESCARTAVEL,228.250833


Shape final: (1448, 4)


In [5]:
import pandas as pd
import numpy as np

# Dataframes dos blocos anteriores:
# df_fin_pnl_final (do Bloco 2, com chave_sku completa)
# df_projecao_final (do Bloco 4, com chave_sku_parcial)

print("1. Enriquecendo a Projeção (SKU)")

# 1. Criar o "Dicionário" de SKUs (P&L Histórico -> Projeção)

# 1a. Criar a chave parcial no P&L Histórico
# (Usamos .loc para evitar o 'SettingWithCopyWarning')
df_fin_pnl_final.loc[:, 'chave_sku_parcial'] = (
    df_fin_pnl_final["chave_sku"].str.split("|").str[0] + "|" + # Marca
    df_fin_pnl_final["chave_sku"].str.split("|").str[1] + "|" + # Tamanho
    df_fin_pnl_final["chave_sku"].str.split("|").str[3]  # Retornabilidade
)

# 1b. Criar o "dicionário" de lookup (SKU Parcial -> SKU Completo)
# Isso mapeia a chave de 3 partes (Projeção) para a chave de 5 partes (Financeiro)
sku_enrichment_map = df_fin_pnl_final.drop_duplicates(subset=['chave_sku_parcial'])
sku_enrichment_map = sku_enrichment_map.set_index('chave_sku_parcial')['chave_sku']

print(f"Mapa de enriquecimento de SKU criado. {len(sku_enrichment_map)} SKUs parciais encontrados.")

# 2. Aplicar o Mapa e Agregar a Projeção
print("\n2. Aplicando o mapa à Planilha de Projeção")

df_proj_harmonizada = df_projecao_final.copy()

# 2a. Aplicar mapa de SKU (para obter a chave completa de 5 partes)
df_proj_harmonizada['chave_sku'] = df_proj_harmonizada['chave_sku_parcial'].map(sku_enrichment_map)

# 2b. Filtrar linhas da Projeção que não encontramos no Financeiro
df_proj_filtrada = df_proj_harmonizada.dropna(subset=['chave_sku'])
print(f"Shape da Projeção após enriquecimento de SKU: {df_proj_filtrada.shape}")

# 2c. Agrupar (somar volumes)
# A Projeção não tem 'regional', então agregamos por 'depara_mess', 'diretoria', e 'chave_sku'
final_proj_agg_cols = [
    'depara_mess',
    'diretoria',
    'chave_sku'
]

df_projecao_agg = (
    df_proj_filtrada.groupby(final_proj_agg_cols)
    .agg(volume_projetado=('volume_projetado', 'sum'))
    .reset_index()
)

# 3. Validação final
print("\n3. Projeção Final Harmonizada e Agregada")
display(df_projecao_agg.head())
print(f"Shape final da Projeção: {df_projecao_agg.shape}")

n_skus_pnl = len(df_fin_pnl_final['chave_sku'].unique())
n_skus_proj = len(df_projecao_agg['chave_sku'].unique())
inter_proj = len(set(df_fin_pnl_final['chave_sku']) & set(df_projecao_agg['chave_sku']))

print(f"\nSKUs no P&L Histórico: {n_skus_pnl}")
print(f"SKUs na Projeção Futura: {n_skus_proj}")
print(f"SKUs em Comum: {inter_proj}")

1. Enriquecendo a Projeção (SKU)
Mapa de enriquecimento de SKU criado. 100 SKUs parciais encontrados.

2. Aplicando o mapa à Planilha de Projeção
Shape da Projeção após enriquecimento de SKU: (737, 5)

3. Projeção Final Harmonizada e Agregada


/var/folders/tq/34xgqh111317l5mqqbxpgjxw0000gn/T/ipykernel_48485/3076377947.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin_pnl_final.loc[:, 'chave_sku_parcial'] = (


,depara_mess,diretoria,chave_sku,volume_projetado
0,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.22|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,122313.827291
1,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.25|IMEDIATO (SS)|DESCARTAVEL|VIDRO,4823.883000
2,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.29|IMEDIATO (SS)|RETORNAVEL|VIDRO,62491.135063
3,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.2|IMEDIATO (SS)|DESCARTAVEL|PET,93462.581891
4,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.31|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,17829.536003


Shape final da Projeção: (737, 4)

SKUs no P&L Histórico: 101
SKUs na Projeção Futura: 37
SKUs em Comum: 37


In [6]:
import pandas as pd
import numpy as np

# Dataframes dos blocos anteriores:
# df_projecao_agg (do Bloco 5)
# df_elasticidade_final (do Bloco 3)

print("1. Juntando Projeção de Volume + Elasticidade")

# Usar df_projecao_agg como base (esquerda)
df_master_sim = pd.merge(
    df_projecao_agg,
    df_elasticidade_final,
    on="chave_sku",
    how="left" # Manter todas as 737 linhas de projeção (do Bloco 5)
)

# 2. Análise de Cobertura da Elasticidade
n_total_linhas = len(df_master_sim)
n_sem_elasticidade = df_master_sim['elasticidade'].isna().sum()

print(f"Linhas de Projeção (total): {n_total_linhas}")
print(f"Linhas com Elasticidade: {n_total_linhas - n_sem_elasticidade}")
print(f"Linhas SEM Elasticidade (NaN): {n_sem_elasticidade}")

# 3. Preenchimento de Proxy (Imputação)
if n_sem_elasticidade > 0:
    # Calcular um valor "proxy" (a mediana) para SKUs sem elasticidade
    proxy_elasticidade = df_elasticidade_final['elasticidade'].median()
    if pd.isna(proxy_elasticidade):
        proxy_elasticidade = -1.3 # Backup
        
    print(f"\nPreenchendo {n_sem_elasticidade} linhas com elasticidade 'proxy' (mediana): {proxy_elasticidade:.4f}")
    
    # Aplicar o preenchimento
    df_master_sim['elasticidade'] = df_master_sim['elasticidade'].fillna(proxy_elasticidade)
else:
    print("\nSucesso: Todas as linhas de projeção já tinham elasticidade.")

# 4. Resultado (O Motor de "Demanda")
print("\n'Master DataFrame' (Demanda) está pronto")
display(df_master_sim.head())
print(f"Shape final: {df_master_sim.shape}")

1. Juntando Projeção de Volume + Elasticidade
Linhas de Projeção (total): 737
Linhas com Elasticidade: 632
Linhas SEM Elasticidade (NaN): 105

Preenchendo 105 linhas com elasticidade 'proxy' (mediana): -1.3000

'Master DataFrame' (Demanda) está pronto


,depara_mess,diretoria,chave_sku,volume_projetado,elasticidade
0,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.22|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,122313.827291,-1.620531
1,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.25|IMEDIATO (SS)|DESCARTAVEL|VIDRO,4823.883000,-1.300000
2,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.29|IMEDIATO (SS)|RETORNAVEL|VIDRO,62491.135063,-1.470017
3,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.2|IMEDIATO (SS)|DESCARTAVEL|PET,93462.581891,-1.162830
4,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.31|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,17829.536003,-1.366587


Shape final: (737, 5)


In [7]:
import pandas as pd
import numpy as np

# Dataframes dos blocos anteriores:
# df_master_sim (do Bloco 6)
# df_fin_pnl_final (do Bloco 2)
# df_fin_agg (do Bloco 2, ANTES do cálculo unitário)

print("1. Criando o 'P&L Base' (Média Histórica Ponderada)")

# Para calcular a média unitária correta, usar média ponderada
# 'df_fin_agg' (tem os totais brutos)
# Agrupar o Financeiro por SKU (para todo o período histórico)

# Definir TODOS os 7 drivers de custo
COST_DRIVERS_RAW = [
    'cvv_concentrate', 'cvv_sweetener', 'cvv_pet', 'cvv_can',
    'cvv_cap', 'cvv_purcharses', 'cvv_otherraw'
]

# Definir o mapa de agregação
agg_map = {
    'volume': 'sum',
    'gross_revenue': 'sum',
    'net_revenue': 'sum',
    'gvv_labor': 'sum', # Driver de GVV
    'variable_margin': 'sum'
}
# Adicionar todos os 7 drivers ao mapa
for col in COST_DRIVERS_RAW:
    agg_map[col] = 'sum'

df_pnl_base_agg = (
    df_fin_agg.groupby('chave_sku')
    .agg(agg_map)
    .reset_index()
)

# Calcular as métricas unitárias base (média ponderada)
df_pnl_base_unit = df_pnl_base_agg.loc[df_pnl_base_agg['volume'] > 0].copy()

# A. Receita
df_pnl_base_unit['base_preco_bruto_unit'] = df_pnl_base_unit['gross_revenue'] / df_pnl_base_unit['volume']
df_pnl_base_unit['base_preco_liquido_unit'] = df_pnl_base_unit['net_revenue'] / df_pnl_base_unit['volume']

# B. GVV
df_pnl_base_unit['base_gvv_labor_unit'] = df_pnl_base_unit['gvv_labor'] / df_pnl_base_unit['volume']

# C. Margem
df_pnl_base_unit['base_margem_variavel_unit'] = df_pnl_base_unit['variable_margin'] / df_pnl_base_unit['volume']

# Selecionar colunas de 'chave_sku' e as métricas unitárias 'base'
base_unit_cols = [
    'chave_sku', 'base_preco_bruto_unit', 'base_preco_liquido_unit',
    'base_gvv_labor_unit', 'base_margem_variavel_unit'
]

print("\nCalculando métricas unitárias base para todos os 7 drivers...")
for col in COST_DRIVERS_RAW:
    unit_col_name = f'base_{col}_unit'
    df_pnl_base_unit[unit_col_name] = df_pnl_base_unit[col] / df_pnl_base_unit['volume']
    base_unit_cols.append(unit_col_name) # Adicionar à lista final

df_pnl_base_final = df_pnl_base_unit[base_unit_cols]

print(f"P&L Base Histórico criado. Shape: {df_pnl_base_final.shape}")
display(df_pnl_base_final.head())


# 2. Juntar o P&L base ao 'Master DataFrame'
print("\n2. Juntando P&L Base ao 'Master' (Demanda)")

df_master_sim = pd.merge(
    df_master_sim,  # O dataframe do Bloco 6 (737 linhas)
    df_pnl_base_final, # O P&L base que foi criado agora
    on="chave_sku",
    how="left"
)

# 3. Análise de cobertura do P&L base
n_total_linhas_master = len(df_master_sim)
n_sem_pnl = df_master_sim['base_preco_bruto_unit'].isna().sum()

print(f"Linhas de Projeção (total): {n_total_linhas_master}")
print(f"Linhas com P&L Base: {n_total_linhas_master - n_sem_pnl}")
print(f"Linhas SEM P&L Base (NaN): {n_sem_pnl}")

if n_sem_pnl > 0:
    print(f"ATENÇÃO: {n_sem_pnl} linhas da Projeção Futura são de SKUs que NÃO existem no P&L Histórico.")
    # Remover essas linhas, pois não é possível simular sem um P&L base
    df_master_sim = df_master_sim.dropna(subset=['base_preco_bruto_unit'])
    print(f"Linhas órfãs removidas. Shape final: {df_master_sim.shape}")
else:
    print("\nSucesso: Todas as linhas de projeção têm um P&L Base.")


print("\n'Master DataFrame' (Demanda + P&L Base) está pronto")
display(df_master_sim.head())
print(f"Shape final: {df_master_sim.shape}")

1. Criando o 'P&L Base' (Média Histórica Ponderada)

Calculando métricas unitárias base para todos os 7 drivers...
P&L Base Histórico criado. Shape: (99, 12)


,chave_sku,base_preco_bruto_unit,base_preco_liquido_unit,base_gvv_labor_unit,base_margem_variavel_unit,base_cvv_concentrate_unit,base_cvv_sweetener_unit,base_cvv_pet_unit,base_cvv_can_unit,base_cvv_cap_unit,base_cvv_purcharses_unit,base_cvv_otherraw_unit
0,CC CAFE|0.22|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,4.670209,2.990393,0.161026,0.726070,0.612954,0.073098,0.0,1.076986,0.074477,0.000000,0.133536
1,CC IMPORT|0.35|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,4.474488,3.919945,0.109150,0.301437,0.011433,0.000000,0.0,0.000000,0.000000,3.338136,0.000000
2,CC IMPORT|1.0|FUTURO (MS)|RETORNAVEL|VIDRO,1.991908,1.747864,0.127750,0.122354,0.000000,0.000000,0.0,0.000000,0.000000,1.204715,0.000000
3,CC IMPORT|2.0|FUTURO (MS)|DESCARTAVEL|PET,2.464338,2.165551,0.058742,0.309596,0.000000,0.000000,0.0,0.000000,0.000000,1.673146,0.000000
4,CC MA IMPORT|2.0|FUTURO (MS)|DESCARTAVEL|PET,2.457657,2.073457,0.065834,0.381958,0.000000,0.000000,0.0,0.000000,0.000000,1.521334,0.000000



2. Juntando P&L Base ao 'Master' (Demanda)
Linhas de Projeção (total): 737
Linhas com P&L Base: 716
Linhas SEM P&L Base (NaN): 21
ATENÇÃO: 21 linhas da Projeção Futura são de SKUs que NÃO existem no P&L Histórico.
Linhas órfãs removidas. Shape final: (716, 16)

'Master DataFrame' (Demanda + P&L Base) está pronto


,depara_mess,diretoria,chave_sku,volume_projetado,elasticidade,base_preco_bruto_unit,base_preco_liquido_unit,base_gvv_labor_unit,base_margem_variavel_unit,base_cvv_concentrate_unit,base_cvv_sweetener_unit,base_cvv_pet_unit,base_cvv_can_unit,base_cvv_cap_unit,base_cvv_purcharses_unit,base_cvv_otherraw_unit
0,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.22|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,122313.827291,-1.620531,4.784256,2.968161,0.132349,0.806815,0.476844,0.160081,0.000000,1.075429,0.076264,0.0,0.096860
1,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.25|IMEDIATO (SS)|DESCARTAVEL|VIDRO,4823.883000,-1.300000,6.095929,3.956624,0.139898,0.858011,0.627102,0.161502,1.497588,0.000000,0.064028,0.0,0.351942
2,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.29|IMEDIATO (SS)|RETORNAVEL|VIDRO,62491.135063,-1.470017,3.919290,2.173486,0.138364,0.970938,0.340502,0.160231,0.000000,0.000000,0.052967,0.0,0.054927
3,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.2|IMEDIATO (SS)|DESCARTAVEL|PET,93462.581891,-1.162830,3.938560,2.302362,0.136218,0.715281,0.366573,0.160244,0.596268,0.000000,0.078357,0.0,0.085470
4,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.31|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,17829.536003,-1.366587,4.602462,2.796943,0.132572,0.741269,0.466253,0.157852,0.000000,0.977098,0.061924,0.0,0.126404


Shape final: (716, 16)


In [8]:
import pandas as pd
import numpy as np

# Dataframes dos blocos anteriores:
# df_master_sim (do Bloco 7)
# df_mp_clean (do Bloco 1)

print("1. Preparando o DataFrame Master (criando colunas-helper de merge)")
# Extrair 'tamanho_str' e 'embalagem' da chave_sku para o merge
sku_parts = df_master_sim['chave_sku'].str.split('|', expand=True)
# Usar .loc para evitar o SettingWithCopyWarning
df_master_sim.loc[:, 'marca_base'] = sku_parts[0]
df_master_sim.loc[:, 'tamanho_num_str'] = sku_parts[1] # ex: '1.5', '0.22'
df_master_sim.loc[:, 'embalagem_tipo'] = sku_parts[4] # ex: 'PET', 'ALUMINIO'

print("Colunas-helper ('tamanho_num_str', 'embalagem_tipo') criadas.")

print("\n2. Isolando drivers de custo futuro da aba 'Matéria-Prima'")

# 2a. Driver: PTAX (Câmbio)
df_ptax = df_mp_clean[
    (df_mp_clean['item_materia_prima'] == 'ITEM') & 
    (df_mp_clean['unidade_medida'] == 'PTAX')
].rename(columns={'custo_driver_projetado': 'futuro_ptax'})
df_ptax = df_ptax[['depara_mess', 'futuro_ptax']]
print(f"   Driver PTAX isolado. Shape: {df_ptax.shape}")

# 2b. Driver: Açúcar (Sweetener)
df_sweetener = df_mp_clean[
    df_mp_clean['id_planta'].astype(str).str.upper().str.contains('AÇÚCAR', na=False) 
].rename(columns={'custo_driver_projetado': 'futuro_sweetener_cost'})
# Usando a média de 'Cristal' e 'Refinado' por mês
df_sweetener = df_sweetener.groupby('depara_mess').agg(
    futuro_sweetener_cost=('futuro_sweetener_cost', 'mean')
).reset_index()
print(f"   Driver Açúcar isolado. Shape: {df_sweetener.shape}")

# 2c. Driver: Embalagens (PET, Lata, Vidro, BIB)
df_packaging = df_mp_clean[
    df_mp_clean['id_planta'].astype(str).str.upper().str.contains('GARRAFA|LATA|CX. PAPELÃO', na=False)
].copy()

# Função para traduzir nomes de itens (ex: 'JDLata 7.5oz') para chaves (ex: '0.22', 'ALUMINIO')
def map_driver_item_to_keys(item_name):
    item = str(item_name).upper() 
    
    # Embalagem
    embalagem = np.nan
    if 'PET' in item: embalagem = 'PET'
    elif 'LATA' in item: embalagem = 'ALUMINIO'
    elif 'GARRAFA OWG' in item: embalagem = 'VIDRO'
    elif 'CX. PAPELÃO' in item: embalagem = 'BAG IN BOX'
    
    # Tamanho
    tamanho_str = np.nan
    match_l = re.search(r"(\d+[\.,]\d*)\s*L", item)
    if match_l:
        tamanho_str = match_l.group(1).replace(',', '.') # ex: '1.5'
    else:
        match_ml = re.search(r"(\d+)\s*ML", item)
        if match_ml:
            tamanho_str = str(float(match_ml.group(1)) / 1000.0) # ex: '0.22'
        elif '9,8OZ' in item: tamanho_str = '0.29'
        elif '7.5OZ' in item: tamanho_str = '0.22'
        elif '10.5OZ' in item: tamanho_str = '0.31'
        elif '12OZ' in item: tamanho_str = '0.35'
        
    return pd.Series([tamanho_str, embalagem])

# Aplicar a tradução na coluna 'id_planta' (que tem o nome completo)
df_packaging[['tamanho_num_str', 'embalagem_tipo']] = \
    df_packaging['id_planta'].apply(map_driver_item_to_keys)

df_packaging = df_packaging.rename(columns={'custo_driver_projetado': 'futuro_embalagem_cost'})
df_packaging = df_packaging.dropna(subset=['tamanho_num_str', 'embalagem_tipo'])

# Manter apenas as colunas de merge
df_packaging = df_packaging[[
    'depara_mess', 'tamanho_num_str', 'embalagem_tipo', 'futuro_embalagem_cost'
]].drop_duplicates()

print(f"   Drivers de Embalagem isolados. Shape: {df_packaging.shape}")

print("\n3. Juntando os drivers de custo futuro ao DataFrame Master")
df_master_final = df_master_sim.copy()

# Garantir que todas as chaves 'depara_mess' sejam datetime antes do merge
df_master_final['depara_mess'] = pd.to_datetime(df_master_final['depara_mess'])
df_ptax['depara_mess'] = pd.to_datetime(df_ptax['depara_mess'])
df_sweetener['depara_mess'] = pd.to_datetime(df_sweetener['depara_mess'])
df_packaging['depara_mess'] = pd.to_datetime(df_packaging['depara_mess'])

# 3a. Juntar PTAX (só no mês)
df_master_final = pd.merge(df_master_final, df_ptax, on='depara_mess', how='left')

# 3b. Juntar Açúcar (só no mês)
df_master_final = pd.merge(df_master_final, df_sweetener, on='depara_mess', how='left')

# 3c. Juntar Embalagem (na data, tamanho e tipo de embalagem)
df_master_final = pd.merge(
    df_master_final, 
    df_packaging, 
    on=['depara_mess', 'tamanho_num_str', 'embalagem_tipo'],
    how='left'
)

print("\n4. Aplicando regras de negócio pós-merge (ex: Custo Zero Açúcar)")
# Regra 1: SKUs 'ZERO' ou 'SEM AÇÚ' não têm custo de açúcar
zero_sugar_mask = df_master_final['chave_sku'].str.contains('ZERO|SEM AÇÚ')
df_master_final.loc[zero_sugar_mask, 'futuro_sweetener_cost'] = 0

# Regra 2: Preencher NaNs (embalagens não mapeadas) com 0
df_master_final['futuro_embalagem_cost'] = df_master_final['futuro_embalagem_cost'].fillna(0)
# Regra 3: Preencher NaNs (açúcar para SKUs não-zero) com 0
df_master_final['futuro_sweetener_cost'] = df_master_final['futuro_sweetener_cost'].fillna(0)
# Regra 4: Preencher PTAX para todos os meses (forward-fill e back-fill)
df_master_final['futuro_ptax'] = df_master_final['futuro_ptax'].ffill().bfill() # Corrigido o ffill warning

# Limpar colunas helper
df_master_final = df_master_final.drop(
    columns=['marca_base', 'tamanho_num_str', 'embalagem_tipo']
)

print("\n'Master DataFrame' (Completo) está pronto")
display(df_master_final.head())
print(f"Shape final: {df_master_final.shape}")
print("\nVerificação de NaNs nos custos futuros:")
print(df_master_final[['futuro_embalagem_cost', 'futuro_sweetener_cost', 'futuro_ptax']].isna().sum())

1. Preparando o DataFrame Master (criando colunas-helper de merge)
Colunas-helper ('tamanho_num_str', 'embalagem_tipo') criadas.

2. Isolando drivers de custo futuro da aba 'Matéria-Prima'
   Driver PTAX isolado. Shape: (12, 2)
   Driver Açúcar isolado. Shape: (12, 2)
   Drivers de Embalagem isolados. Shape: (108, 4)

3. Juntando os drivers de custo futuro ao DataFrame Master

4. Aplicando regras de negócio pós-merge (ex: Custo Zero Açúcar)

'Master DataFrame' (Completo) está pronto


,depara_mess,diretoria,chave_sku,volume_projetado,elasticidade,base_preco_bruto_unit,base_preco_liquido_unit,base_gvv_labor_unit,base_margem_variavel_unit,base_cvv_concentrate_unit,base_cvv_sweetener_unit,base_cvv_pet_unit,base_cvv_can_unit,base_cvv_cap_unit,base_cvv_purcharses_unit,base_cvv_otherraw_unit,futuro_ptax,futuro_sweetener_cost,futuro_embalagem_cost
0,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.22|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,122313.827291,-1.620531,4.784256,2.968161,0.132349,0.806815,0.476844,0.160081,0.000000,1.075429,0.076264,0.0,0.096860,5.4571,107.355999,19.949874
1,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.25|IMEDIATO (SS)|DESCARTAVEL|VIDRO,4823.883000,-1.300000,6.095929,3.956624,0.139898,0.858011,0.627102,0.161502,1.497588,0.000000,0.064028,0.0,0.351942,5.4571,107.355999,0.000000
2,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.29|IMEDIATO (SS)|RETORNAVEL|VIDRO,62491.135063,-1.470017,3.919290,2.173486,0.138364,0.970938,0.340502,0.160231,0.000000,0.000000,0.052967,0.0,0.054927,5.4571,107.355999,91.191446
3,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.2|IMEDIATO (SS)|DESCARTAVEL|PET,93462.581891,-1.162830,3.938560,2.302362,0.136218,0.715281,0.366573,0.160244,0.596268,0.000000,0.078357,0.0,0.085470,5.4571,107.355999,11.427864
4,2025-06-01,DISTRIBUIDORES DE AREA,CC|0.31|IMEDIATO (SS)|DESCARTAVEL|ALUMINIO,17829.536003,-1.366587,4.602462,2.796943,0.132572,0.741269,0.466253,0.157852,0.000000,0.977098,0.061924,0.0,0.126404,5.4571,107.355999,23.632022


Shape final: (716, 19)

Verificação de NaNs nos custos futuros:
futuro_embalagem_cost    0
futuro_sweetener_cost    0
futuro_ptax              0
dtype: int64


In [9]:
import pandas as pd
import numpy as np

# Dataframes dos blocos anteriores:
# df_master_final (do Bloco 8)
# df_fin_agg (do Bloco 2)
# df_fin_pnl_final (do Bloco 7)

# 1. Corrigir o FutureWarning
df_master_final['futuro_ptax'] = df_master_final['futuro_ptax'].ffill().bfill()
print("1. FutureWarning corrigido")

# 2. Definir os Nomes dos Drivers
COST_DRIVERS_SLIDERS = [
    "cvv_concentrate",
    "cvv_sweetener",
    "cvv_pet",
    "cvv_can",
    "cvv_cap",
    "cvv_purcharses",
    "cvv_otherraw",
]
# nomes das colunas de custo base que do Bloco 7
COST_BUCKETS_BASE_COLS = [f"base_{c}_unit" for c in COST_DRIVERS_SLIDERS]

# 3. A Função de Simulação
def run_pnl_simulation(
    base_df, 
    price_adj_pct=0.0, 
    shocks_pct={}
    ):
    """
    Roda o simulador de P&L com a regra de concentrate como % da receita.
    """
    df_sim = base_df.copy()
    
    # 1. Simular Preço
    df_sim['preco_liquido_simulado_unit'] = df_sim['base_preco_liquido_unit'].fillna(0) * (1 + price_adj_pct)
    
    # 2. Simular Volume (Elasticidade)
    # torna a divisão mais robusta para evitar erros de 0
    df_sim['variacao_preco_pct'] = 0.0
    mask_preco_valido = df_sim['base_preco_liquido_unit'] != 0
    df_sim.loc[mask_preco_valido, 'variacao_preco_pct'] = (
        df_sim.loc[mask_preco_valido, 'preco_liquido_simulado_unit'] / 
        df_sim.loc[mask_preco_valido, 'base_preco_liquido_unit']
    ) - 1
    
    df_sim['variacao_volume_pct'] = df_sim['variacao_preco_pct'] * df_sim['elasticidade']
    df_sim['volume_simulado'] = df_sim['volume_projetado'] * (1 + df_sim['variacao_volume_pct'])

    # 3. Simular Custos Unitários
    
    # Custo Total Base (R$/UC) = Preço Líquido - Margem
    df_sim['custo_total_base_unit'] = df_sim['base_preco_liquido_unit'] - df_sim['base_margem_variavel_unit']
    
    # "Outros Custos" = Custo Total - TODOS os 7 drivers base
    df_sim['outros_custos_unit'] = df_sim['custo_total_base_unit']
    for col in COST_BUCKETS_BASE_COLS: # Iterar sobre os 7 drivers
        df_sim['outros_custos_unit'] -= df_sim[col].fillna(0)

    # 3a. Custo Especial: Concentrate (% da Receita)
    # Calcular o % base (ex: 0.05 para 5%)
    df_sim['perc_concentrate_base'] = 0.0
    df_sim.loc[mask_preco_valido, 'perc_concentrate_base'] = (
        df_sim.loc[mask_preco_valido, 'base_cvv_concentrate_unit'].fillna(0) / 
        df_sim.loc[mask_preco_valido, 'base_preco_liquido_unit']
    )
    
    # Aplicar o "choque" ao percentual (ex: 5% * (1 + 0.10) = 5.5%)
    shock_conc_pct = shocks_pct.get('cvv_concentrate', 0.0)
    df_sim['perc_concentrate_simulado'] = df_sim['perc_concentrate_base'] * (1 + shock_conc_pct)
    
    # Calcular o Custo Simulado (ex: 5.5% * Preço Simulado)
    df_sim['cvv_concentrate_simulado_unit'] = df_sim['preco_liquido_simulado_unit'] * df_sim['perc_concentrate_simulado']
    
    # 3b. Custos Padrão (os 6 restantes)
    # Custo Total Simulado = Outros + (Concentrate Simulado) + (6 Drivers Simulados)
    df_sim['custo_total_simulado_unit'] = df_sim['outros_custos_unit'] + df_sim['cvv_concentrate_simulado_unit']

    # Loop nos 6 drivers padrão
    standard_drivers = [c for c in COST_DRIVERS_SLIDERS if c != 'cvv_concentrate']
    
    for driver_key in standard_drivers: 
        base_cost_col = f'base_{driver_key}_unit'
        shock_pct = shocks_pct.get(driver_key, 0.0) # Pega o 'choque' do dicionário
        sim_cost_col = f'{driver_key}_simulado_unit'
        
        df_sim[sim_cost_col] = df_sim[base_cost_col].fillna(0) * (1 + shock_pct)
        df_sim['custo_total_simulado_unit'] += df_sim[sim_cost_col] # Adiciona os 6 simulados

    # 4. Calcular P&L Simulado
    df_sim['margem_simulada_unit'] = (
        df_sim['preco_liquido_simulado_unit'] - df_sim['custo_total_simulado_unit']
    )
    
    # 5. Calcular Totais
    df_sim['receita_total_simulada'] = df_sim['volume_simulado'] * df_sim['preco_liquido_simulado_unit']
    df_sim['margem_total_simulada'] = df_sim['volume_simulado'] * df_sim['margem_simulada_unit']
    
    return df_sim

# 4. TESTE: Rodar a Simulação

print("\n3. Rodando Simulações de Teste")

# Cenário 1: Base (sem choques)
df_cenario_base = run_pnl_simulation(df_master_final, shocks_pct={})

# Cenário 2: Aumento de 10% no preço de tudo
df_cenario_preco_10 = run_pnl_simulation(
    df_master_final,
    price_adj_pct=0.10,
    shocks_pct={}
)

# Cenário 3: Aumento de 10% no preço E +20% no custo do PET
df_cenario_preco_custo = run_pnl_simulation(
    df_master_final,
    price_adj_pct=0.10,
    shocks_pct={'cvv_pet': 0.20} # O 'shock' agora é um dicionário
)

# Cenário 4: Teste da regra do Concentrate (+10% no Preço, +5% no Custo do Concentrate)
df_cenario_concentrate = run_pnl_simulation(
    df_master_final,
    price_adj_pct=0.10,
    shocks_pct={'cvv_concentrate': 0.05}
)


# 5. Comparar os resultados
print("\n4. Resultados Agregados da Simulação")

def summarize_pnl(df, scenario_name):
    total_receita = df['receita_total_simulada'].sum()
    total_margem = df['margem_total_simulada'].sum()
    total_volume = df['volume_simulado'].sum()
    
    # Cálculo seguro da Margem %
    margem_pct = (total_margem / total_receita) if total_receita != 0 else 0
    
    return pd.Series({
        'Cenário': scenario_name,
        'Receita Total': f"R$ {total_receita:,.0f}",
        'Margem Total': f"R$ {total_margem:,.0f}",
        'Volume Total': f"{total_volume:,.0f} UC",
        'Margem %': f"{margem_pct:.2%}"
    })

# Coletar resultados
resultados = [
    summarize_pnl(df_cenario_base, "1. Cenário Base"),
    summarize_pnl(df_cenario_preco_10, "2. Preço +10%"),
    summarize_pnl(df_cenario_preco_custo, "3. Preço +10%, PET +20%"),
    summarize_pnl(df_cenario_concentrate, "4. Preço +10%, Concentrate +5%")
]

df_resultados = pd.DataFrame(resultados).set_index('Cenário')
display(df_resultados)

1. FutureWarning corrigido

3. Rodando Simulações de Teste

4. Resultados Agregados da Simulação


,Receita Total,Margem Total,Volume Total,Margem %
Cenário,,,,
1. Cenário Base,"R$ 200,676,138","R$ 77,308,612","120,369,968 UC",38.52%
2. Preço +10%,"R$ 189,112,149","R$ 80,752,225","103,526,969 UC",42.70%
"3. Preço +10%, PET +20%","R$ 189,112,149","R$ 77,799,053","103,526,969 UC",41.14%
"4. Preço +10%, Concentrate +5%","R$ 189,112,149","R$ 79,230,693","103,526,969 UC",41.90%


In [10]:
import pandas as pd
import numpy as np

# 1. Definir o nome do arquivo de saída
OUTPUT_FILE_NAME = "simulador_pnl_futuro_base.csv"

try:
    print(f"Salvando o 'Master DataFrame' ({df_master_final.shape})")
    
    # Salvar o arquivo final que alimenta o Dash
    # Usa decimal=',' (vírgula) para manter o padrão
    df_master_final.to_csv(OUTPUT_FILE_NAME, index=False, encoding='utf-8', decimal=',')
    
    print(f"SUCESSO: Arquivo salvo como '{OUTPUT_FILE_NAME}'")
    
except Exception as e:
    print(f"ERRO ao salvar o arquivo: {e}")

Salvando o 'Master DataFrame' ((716, 19))
SUCESSO: Arquivo salvo como 'simulador_pnl_futuro_base.csv'
